In [ ]:
# Excel Comparison Analysis

## Import Required Libraries
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path

## Load the Excel Files
# Replace these paths with your actual file paths
extract_file_path = "extract.xlsx"  # Excel 1 (Extract)
gold_file_path = "gold.xlsx"        # Excel 2 (Gold)

# Load Excel files
df_extract = pd.read_excel(extract_file_path)
df_gold = pd.read_excel(gold_file_path)

# Display first few rows of each DataFrame to verify loaded correctly
print("Extract DataFrame Preview:")
display(df_extract.head())

print("Gold DataFrame Preview:")
display(df_gold.head())

## Data Preparation

# Function to extract INSTRUMENT_ID from filepath
def extract_instrument_id(filepath):
    # Extract filename from path and remove file extension
    if pd.isna(filepath):
        return None
    try:
        # Extract the last part of the filepath before the underscore and file extension
        pattern = r'([^/\\]+)_f\.jpeg$'
        match = re.search(pattern, filepath)
        if match:
            return match.group(1)
        return None
    except:
        return None

# Apply the function to extract INSTRUMENT_ID
df_extract['extracted_instrument_id'] = df_extract['filepath'].apply(extract_instrument_id)

# Display the extracted instrument IDs
print("\nExtracted Instrument IDs:")
display(df_extract[['filepath', 'extracted_instrument_id']].head())

## Merge DataFrames for Comparison
# Merge the extract and gold DataFrames on instrument ID
merged_df = pd.merge(
    df_extract, 
    df_gold, 
    left_on='extracted_instrument_id', 
    right_on='INSTRUMENT_ID', 
    how='left',
    suffixes=('_extract', '_gold')
)

# Check the merged DataFrame
print("\nMerged DataFrame Preview:")
display(merged_df.head())
print(f"Total records in extract: {len(df_extract)}")
print(f"Total records after merging: {len(merged_df)}")
print(f"Records with no match in gold: {len(merged_df[merged_df['INSTRUMENT_ID'].isna()])}")

## Comparison Functions

# Function to clean text for comparison
def clean_text(text):
    if pd.isna(text):
        return ""
    # Convert to string, lowercase, remove leading/trailing spaces and special characters
    return re.sub(r'[^a-zA-Z0-9]', '', str(text).lower().strip())

# Function to clean and compare amounts
def compare_amounts(amount1, amount2):
    if pd.isna(amount1) and pd.isna(amount2):
        return True
    if pd.isna(amount1) or pd.isna(amount2):
        return False
    
    # Convert to float for numeric comparison
    try:
        float1 = float(amount1)
        float2 = float(amount2)
        return float1 == float2
    except:
        return False

# Function to concatenate bank fields
def concatenate_bank_fields(city_code, bank_code, branch_code):
    # Handle NaN values
    city_code = "" if pd.isna(city_code) else str(city_code)
    bank_code = "" if pd.isna(bank_code) else str(bank_code)
    branch_code = "" if pd.isna(branch_code) else str(branch_code)
    
    return f"{city_code}{bank_code}{branch_code}"

## Perform Comparisons

# Create a copy of merged DataFrame for results
results_df = merged_df.copy()

# Create comparison columns
results_df['instrument_number_match'] = merged_df.apply(
    lambda row: clean_text(row['micr_scan_instrument_number']) == clean_text(row['SCAN_INSTRUMENT_NUMBER']), 
    axis=1
)

results_df['payee_bank_match'] = merged_df.apply(
    lambda row: clean_text(row['micr_scan_payee_bank']) == clean_text(row['SCAN_PAYEE_BANK_CODE']), 
    axis=1
)

# Concatenate bank fields and compare
results_df['gold_payee_details'] = merged_df.apply(
    lambda row: concatenate_bank_fields(
        row['SCAN_PAYEE_BANK_CITY_CODE'], 
        row['SCAN_PAYEE_BANK_CODE'], 
        row['SCAN_PAYEE_BANK_BRANCH_CODE']
    ), 
    axis=1
)

results_df['payee_details_match'] = results_df.apply(
    lambda row: clean_text(row['micr_scan_payee_details']) == clean_text(row['gold_payee_details']), 
    axis=1
)

results_df['micr_acno_match'] = merged_df.apply(
    lambda row: clean_text(row['micr_scan_micr_acno']) == clean_text(row['SCAN_MICR_ACNO']), 
    axis=1
)

results_df['instrument_type_match'] = merged_df.apply(
    lambda row: clean_text(row['micr_scan_instrument_type']) == clean_text(row['SCAN_INSTRUMENT_TYPE']), 
    axis=1
)

results_df['payee_name_match'] = merged_df.apply(
    lambda row: clean_text(row['payee_name']) == clean_text(row['PRES_NAME']), 
    axis=1
)

results_df['amount_match'] = merged_df.apply(
    lambda row: compare_amounts(row['amount_numeric'], row['CAR_AMOUNT']), 
    axis=1
)

# Calculate overall match for each record (True if all fields match)
results_df['overall_match'] = results_df.apply(
    lambda row: (
        row['instrument_number_match'] and
        row['payee_bank_match'] and
        row['payee_details_match'] and
        row['micr_acno_match'] and
        row['instrument_type_match'] and
        row['payee_name_match'] and
        row['amount_match']
    ),
    axis=1
)

## Calculate Match Percentages
match_stats = {
    'Field': [
        'Instrument Number',
        'Payee Bank',
        'Payee Details',
        'MICR Account Number',
        'Instrument Type',
        'Payee Name',
        'Amount',
        'Overall'
    ],
    'Match Count': [
        results_df['instrument_number_match'].sum(),
        results_df['payee_bank_match'].sum(),
        results_df['payee_details_match'].sum(),
        results_df['micr_acno_match'].sum(),
        results_df['instrument_type_match'].sum(),
        results_df['payee_name_match'].sum(),
        results_df['amount_match'].sum(),
        results_df['overall_match'].sum()
    ],
    'Total Records': len(results_df),
    'Match Percentage': []
}

# Calculate percentages
for count in match_stats['Match Count']:
    if match_stats['Total Records'] > 0:
        percentage = (count / match_stats['Total Records']) * 100
    else:
        percentage = 0
    match_stats['Match Percentage'].append(f"{percentage:.2f}%")

# Create summary DataFrame
summary_df = pd.DataFrame(match_stats)
display(summary_df)

## Detailed Match Analysis
# Create a more detailed output with all match columns
output_columns = [
    'extracted_instrument_id',
    'INSTRUMENT_ID',
    'micr_scan_instrument_number',
    'SCAN_INSTRUMENT_NUMBER',
    'instrument_number_match',
    'micr_scan_payee_bank',
    'SCAN_PAYEE_BANK_CODE',
    'payee_bank_match',
    'micr_scan_payee_details',
    'gold_payee_details',
    'payee_details_match',
    'micr_scan_micr_acno',
    'SCAN_MICR_ACNO',
    'micr_acno_match',
    'micr_scan_instrument_type',
    'SCAN_INSTRUMENT_TYPE',
    'instrument_type_match',
    'payee_name',
    'PRES_NAME',
    'payee_name_match',
    'amount_numeric',
    'CAR_AMOUNT',
    'amount_match',
    'overall_match'
]

detailed_output = results_df[output_columns]

# Display detailed output
print("\nDetailed Match Analysis:")
display(detailed_output.head())

## Save Results to Excel
# Prepare the output Excel file
output_file = "comparison_results.xlsx"

# Create a writer object
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Write the summary sheet
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    # Write the detailed results sheet
    detailed_output.to_excel(writer, sheet_name='Detailed Results', index=False)
    
    # Write mismatches sheet (records where overall_match is False)
    mismatches = detailed_output[~detailed_output['overall_match']]
    mismatches.to_excel(writer, sheet_name='Mismatches', index=False)
    
    # Write matches sheet (records where overall_match is True)
    matches = detailed_output[detailed_output['overall_match']]
    matches.to_excel(writer, sheet_name='Matches', index=False)

print(f"\nResults saved to {output_file}")